In [35]:
from transformers import BartTokenizer, BartForConditionalGeneration
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
import torch
import functools
from transformers import AdamW, get_linear_schedule_with_warmup
import math
from time import time
from datasets import load_dataset, Dataset

In [5]:
path_train = "/Users/datle/Downloads/ELI5.jsonl"
path_val = "/Users/datle/Downloads/ELI5_val.jsonl"
dataset_train = load_dataset('json', data_files = path_train)
dataset_val = load_dataset('json', data_files = path_val)
train = dataset_train['train'].select(range(1,30000))
val = dataset_val['train']

Found cached dataset json (/Users/datle/.cache/huggingface/datasets/json/default-0913b0ed92067c10/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset json (/Users/datle/.cache/huggingface/datasets/json/default-f1b40a908fd17224/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

In [45]:
def preprocess(ex):
    return tokenizer(ex['ques_ctxs'],max_length=128, padding='max_length', truncation=True, return_tensors='pt')

In [29]:
dct ={'question_id':[], 'answer':[]}

In [30]:
def replace_text(string):
    return string.replace("\'","").replace("\n","").replace("URL_0","").lower().strip()
def concat(ex, n):
    ex['question'] = replace_text(ex['question'])
#     context = [k[0] for k in ex['ctxs'][:3]]
#     context = replace_text(' '.join(context))
    context = ex['ctxs'][:n]
    if type((context[0])) == list:
        context = [k[0] for k in context]
    context = replace_text(' '.join(context))
    ex['ctxs'] = context
    ex['ques_ctxs'] = f"question: {ex['question']} context: {context}"
    return ex
def answers(ex):
    for answer in ex['answers']:
        dct['question_id'].append(ex['question_id'])
        dct['answer'].append(answer)

In [42]:
question_ds = train.map(lambda x: concat(x, n=2), remove_columns = ['question', 'answers', 'ctxs'])

Map:   0%|          | 0/29999 [00:00<?, ? examples/s]

In [36]:
answer_ds = Dataset.from_dict(dct)

In [37]:
answer_ds

Dataset({
    features: ['question_id', 'answer'],
    num_rows: 92273
})

In [39]:
def preprocess_answer(ex):
    return tokenizer(ex['answer'],max_length=128, padding='max_length', truncation=True, return_tensors='pt')

In [40]:
tokenized_answer_ds = answer_ds.map(preprocess_answer)

Map:   0%|          | 0/92273 [00:00<?, ? examples/s]

In [47]:
tokenized_question_ds = question_ds.map(preprocess)

Map:   0%|          | 0/29999 [00:00<?, ? examples/s]

In [48]:
tokenized_question_ds

Dataset({
    features: ['question_id', 'ques_ctxs', 'input_ids', 'attention_mask'],
    num_rows: 29999
})

In [51]:
tokenized_answer_ds

Dataset({
    features: ['question_id', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 92273
})

In [53]:
dct_answers ={}
def convert_back(ex):
    if ex['question_id'] not in dct_answers:
        dct_answers[ex['question_id']] = []
        dct_answers[ex['question_id']].append([ex['input_ids'], ex['attention_mask']])
    else:
        dct_answers[ex['question_id']].append([ex['input_ids'], ex['attention_mask']])

In [54]:
tokenized_answer_ds.map(convert_back)

Map:   0%|          | 0/92273 [00:00<?, ? examples/s]

Dataset({
    features: ['question_id', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 92273
})

In [107]:
len(train[25]['answers'])

6

In [59]:
qa_list=[(i,j)
for i,key in enumerate(dct_answers)
for j, answer in enumerate(dct_answers[key])
        ]

In [60]:
qa_list

[(0, 0),
 (0, 1),
 (0, 2),
 (1, 0),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (3, 0),
 (4, 0),
 (5, 0),
 (5, 1),
 (6, 0),
 (6, 1),
 (7, 0),
 (8, 0),
 (9, 0),
 (9, 1),
 (10, 0),
 (10, 1),
 (10, 2),
 (11, 0),
 (12, 0),
 (12, 1),
 (13, 0),
 (13, 1),
 (14, 0),
 (14, 1),
 (15, 0),
 (16, 0),
 (16, 1),
 (17, 0),
 (18, 0),
 (19, 0),
 (19, 1),
 (20, 0),
 (20, 1),
 (21, 0),
 (21, 1),
 (21, 2),
 (21, 3),
 (21, 4),
 (21, 5),
 (21, 6),
 (22, 0),
 (22, 1),
 (23, 0),
 (23, 1),
 (23, 2),
 (24, 0),
 (24, 1),
 (25, 0),
 (25, 1),
 (25, 2),
 (25, 3),
 (25, 4),
 (25, 5),
 (26, 0),
 (26, 1),
 (27, 0),
 (28, 0),
 (28, 1),
 (28, 2),
 (28, 3),
 (28, 4),
 (28, 5),
 (28, 6),
 (28, 7),
 (28, 8),
 (28, 9),
 (28, 10),
 (28, 11),
 (28, 12),
 (28, 13),
 (29, 0),
 (30, 0),
 (30, 1),
 (30, 2),
 (30, 3),
 (31, 0),
 (31, 1),
 (32, 0),
 (32, 1),
 (33, 0),
 (33, 1),
 (33, 2),
 (33, 3),
 (34, 0),
 (34, 1),
 (35, 0),
 (35, 1),
 (35, 2),
 (36, 0),
 (36, 1),
 (36, 2),
 (36, 3),
 (37, 0),
 (37, 1),
 (38, 0),
 (38, 

In [94]:
class eli5dataset(Dataset):
    def __init__(self, data_ques, data_answer):
        self.data_question = data_ques
        self.data_answer = data_answer
        self.qa_id_list = [
            (i,j)
            for i, key in enumerate(self.data_answer)
            for j, a in enumerate(self.data_answer[key])
        ]
    def __len__(self):
        return len(self.qa_id_list)
    def make_example(self, idx):
        q_ids = []
        q_mask = []
        a_ids = []
        a_mask = []
        for i in idx:
            i,j = self.qa_id_list[i]
            
            question_id = self.data_question['question_id'][i]
            q_ids.append(self.data_question['input_ids'][i])
            q_mask.append(self.data_question['attention_mask'][i])
            a_id, a_m = self.data_answer[question_id][j]
            a_ids.append(a_id)
            a_mask.append(a_m)
        
        q_ids, q_mask = (
        torch.squeeze(torch.LongTensor(q_ids)),
        torch.squeeze(torch.LongTensor(q_mask))
        )
        
        a_ids, a_mask = (
        torch.squeeze(torch.LongTensor(a_ids)),
        torch.squeeze(torch.LongTensor(a_mask))
        )

        labels = a_ids[:, 1:].contiguous().clone()
        labels[a_mask[:, 1:].contiguous() == 0] = -100

        model_inputs = {
        'input_ids': q_ids,
        'attention_mask': q_mask,
        'decoder_input_ids': a_ids[:, :-1].contiguous(),
        'labels': labels,
    }
        return model_inputs
    def __getitem__(self, idx):
        print(idx)
        return self.make_example(idx)

In [95]:
def data_loader(dataset, args):
    train_sampler = SequentialSampler(dataset)
    data_loader = DataLoader(dataset, batch_size=args.batch_size, sampler=train_sampler)
    return data_loader

In [96]:
class ArgumentsS2S():
    def __init__(self):
        self.batch_size = 2
        self.max_length = 128

s2s_args = ArgumentsS2S()

In [97]:
!pip install lightning

  Using cached lightning-2.0.4-py3-none-any.whl (1.8 MB)
  Using cached fsspec-2023.6.0-py3-none-any.whl (163 kB)
  Using cached starlette-0.28.0-py3-none-any.whl (68 kB)
  Using cached uvicorn-0.22.0-py3-none-any.whl (58 kB)
     |                                | 358 kB 797 kB/s eta 0:03:00^C

ERROR: Operation cancelled by user


In [98]:
import lightning as L
class bart_model(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
    def forward(self, batch_input):
        output = self.model(**batch_input)
        return output.loss, output.logits
    def training_step(self, batch, batch_idx):
        loss, output = self(batch)
        self.log('train_loss',loss, prog_bar=True, logger=True)
        return loss
    def validation_step(self, batch, batch_idx):
        loss, output = self(batch)
        self.log('val_loss',loss, prog_bar=True, logger=True)
        return loss
    def test_step(self, batch, batch_idx):
        loss, output = self(batch)
        self.log('test_loss',loss, prog_bar=True, logger=True)
        return loss
    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=2e-4)

In [99]:
train2 = eli5dataset(tokenized_question_ds, dct_answers)

train_1 = data_loader(train2, s2s_args)


In [100]:
my_model = bart_model()

In [101]:
trainer = L.Trainer(max_epochs=3)
trainer.fit(my_model, train_1)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type                         | Params
-------------------------------------------------------
0 | model | BartForConditionalGeneration | 139 M 
-------------------------------------------------------
139 M     Trainable params
0         Non-trainable params
139 M     Total params
557.682   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

[0, 1]
torch.Size([2, 128])
[2, 3]
torch.Size([2, 128])
[4, 5]
torch.Size([2, 128])
[6, 7]
torch.Size([2, 128])


/Users/datle/opt/anaconda3/envs/tensorflow/lib/python3.8/site-packages/lightning/pytorch/trainer/call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
